In [8]:
from glob import glob
import os.path as osp
import string
t = [string.atoi(e.split('/')[-1].split('_')[0]) for e in glob('/home/grg/tbss_3x65/origdata/*nii.gz')]
with open('/tmp/tbss_3x65_subjects.txt','w') as w:
    for each in t:
        w.write('%s\n'%each)
    

In [40]:
%run /home/grg/git/alfa/roicollect.py

df = pd.read_excel('/tmp/MMSE_data.xlsx').set_index('ID_Code')

subj = [string.atoi(e) for e in open('/tmp/tbss_3x65_subjects.txt').read().split('\n') if e != '']
cov = pd.read_excel('/tmp/covariates.xls')
cov = cov[cov['subject'].isin(subj)].set_index('subject')
cov = cov.join(df)
df = pd.read_excel('/tmp/Cognition_Data_MRI.xlsx').set_index('ID_Code')
cov = cov.join(df)
cov = cov.dropna()

grp = get_groups(cov, ['HO', 'HT', 'NC'], by='apoe')
cov.head()

,age,apoe,gender,education,MMSE,AGE_MRI,AGE_Cognition,Education_Years,Sex (0 = Female),TIV,...,TFR,TDFR,TDPR,WAIS_DSST,WAIS_Visual_Puzzle,WAIS_Digit_Span_Forward,WAIS_Digit_Span_Backward,WAIS_Digit_Span_Sequence,WAIS_Matrices,WAIS_Similarities
subject,,,,,,,,,,,,,,,,,,,,,
10013,55.414100,2,0,11,30,55.0,54.0,11.0,0.0,1508.795,...,17.0,18.0,26.0,44.0,15.0,14.0,12.0,10.0,14.0,26.0
10024,49.401780,2,0,18,28,49.0,48.0,18.0,0.0,1230.793,...,23.0,24.0,22.0,71.0,18.0,11.0,10.0,9.0,19.0,27.0
10029,50.433949,3,0,17,30,50.0,49.0,17.0,0.0,1641.869,...,26.0,22.0,32.0,76.0,20.0,8.0,12.0,8.0,19.0,29.0
10032,55.386721,2,0,10,28,55.0,53.0,10.0,0.0,1541.632,...,12.0,14.0,17.0,44.0,16.0,7.0,8.0,9.0,16.0,22.0
10034,49.275838,3,0,18,30,49.0,48.0,18.0,0.0,1446.857,...,13.0,23.0,28.0,86.0,16.0,7.0,7.0,9.0,15.0,31.0


In [33]:
from scipy import stats
st = []
d = grp[2]
for each in d.columns:
    st.append((each, len(d[each]), d[each].mean(), d[each].std()))
st = pd.DataFrame(st, columns=['name', 'N', 'mean', 'std'])
st

,name,N,mean,std
0,age,65,54.746654,6.205215
1,apoe,65,2.000000,0.000000
2,gender,65,0.292308,0.458362
3,education,65,13.584615,3.495051
4,MMSE,65,29.169231,1.024226
5,AGE_MRI,65,54.215385,6.273686
6,AGE_Cognition,65,53.184615,6.353868
7,Education_Years,65,13.584615,3.495051
8,Sex (0 = Female),65,0.292308,0.458362
9,TIV,65,1479.914938,169.990833


In [26]:
(len(d[d['gender']==0]), len(d[d['gender']==1]))

(46, 19)

In [27]:
chi2, p, df, _ = stats.chi2_contingency([[19,31,24], [46,34,41]])
chi2,p 

(4.7475988385079289, 0.09312622767761658)

In [42]:
each = 'TFR'
stats.f_oneway(grp[0][each], grp[1][each], grp[2][each])


F_onewayResult(statistic=0.41745308434257539, pvalue=0.65932508392215206)

In [39]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(cov)

,age,apoe,gender,education,MMSE,AGE_MRI,AGE_Cognition,Education_Years,Sex (0 = Female),TIV,...,TFR,TDFR,TDPR,WAIS_DSST,WAIS_Visual_Puzzle,WAIS_Digit_Span_Forward,WAIS_Digit_Span_Backward,WAIS_Digit_Span_Sequence,WAIS_Matrices,WAIS_Similarities
subject,,,,,,,,,,,,,,,,,,,,,
11850,53.242984,3,1,15,29,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66257,65.875428,4,0,10,30,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
